In [18]:
# Copyright 2024 Forusone
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agent Engine in Vertex AI - Prompt template


This colab explains how to deal with prompt template for better query and reasoning

References :  
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/develop#prompt-template
*  https://python.langchain.com/v0.2/docs/concepts/#prompt-templates



## Init Vertex AI SDK

In [19]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[agent_engines,langchain,ag2]

In [20]:
# Note:  Need to restart the kernel
%pip install  --user --quiet google-cloud-firestore
%pip install  --user --quiet langchain-google-firestore
%pip install  --user --quiet langchain-google-vertexai

In [21]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
#MODEL_NAME = "gemini-1.5-flash-002"
MODEL_NAME = "gemini-2.0-flash-001"

# Firestore
DATABASE="chat-history"
COLLECTION="history"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Configuration and initialization

In [22]:
# @title Create a bucket.
BUCKET_URI = f"gs://agent-0319"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0319/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0319' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [23]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [24]:
# @title Set access to the bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://agent-0319/
No changes made to gs://agent-0319/


### Initialize Vertex AI

In [25]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Firestore setting for chat history

### Helper method to get session history

In [26]:

# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    firestore_client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    return FirestoreChatMessageHistory(
        client=firestore_client,
        session_id=session_id,
        collection=COLLECTION,
        encode_message=False,
    )

def get_converstion(chat_history):
  """
  Get conversation from chat history
  """

  from langchain.schema import HumanMessage, AIMessage
  import langchain_core.messages

  for message in chat_history.messages:
      if type(message) is HumanMessage:
        print(f"Human : {message.content}")
      elif type(message) is AIMessage:
        print(f"AI : {message.content}")

def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']


### Unit test of Firestore

In [27]:
# Tool define
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
  """
     지정된 날짜의 두 통화 간 환율을 검색합니다.
  """

  import requests

  response = requests.get(
      f"https://api.frankfurter.app/{currency_date}",
      params={"from": currency_from, "to": currency_to},
  )
  return response.json()

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents.format_scratchpad.tools import format_to_tool_messages

custom_prompt_template = {
    "user_input": lambda x: x["input"],
    "history": lambda x: x["history"],
    "agent_scratchpad": lambda x: format_to_tool_messages(x["intermediate_steps"]),
} | ChatPromptTemplate.from_messages([
    ("placeholder", "{history}"),
    ("user", "{user_input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [29]:
from time import perf_counter
from vertexai.preview.reasoning_engines import LangchainAgent

session_id = "conv-005"

agent = LangchainAgent(
    model=MODEL_NAME,
    prompt=custom_prompt_template,
    chat_history=get_session_history,
    tools=[get_exchange_rate],
)

In [30]:
while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(agent, query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"Elapse time : {t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
print(chat_history.messages)

사용자: 안녕하세요.


Elapse time : 1.6974635740000394

AI Agent : 무엇을 도와드릴까요?

------------------------------------ 
사용자: 오늘 증시 상황이 어떤가요 ?


Elapse time : 1.1144785989999946

...

AI Agent : 환율과 관련된 질문에만 답변할 수 있습니다. 다른 질문은 답변할 수 없습니다.

------------------------------------ 
사용자: 왜 그렇죠 ?


Elapse time : 1.131120391999957

AI Agent : 저는 환율 정보를 제공하도록 설계되었기 때문입니다. 다른 기능은 아직 사용할 수 없습니다.


------------------------------------ 
사용자: 종료
[]
